In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz

# CODE

## 1. Pre-processing

In [2]:
# parameters definition
parent_chain = 'booker' # lower case and "clean"
parent_chain_column = 'parent_chain_name'
item_column = 'sku_name'
language_ = 'en'
threshold_ = 82
parent_chain_use = True

In [3]:
# reading raw data
data = pd.read_csv('UK_products.csv')

In [4]:
def clean_text(df, col_name, new_col_name):
    # column values to lower case
    df[new_col_name] = df[col_name].str.lower().str.strip()
    # removes special characters
    df[new_col_name] = df[new_col_name].apply(lambda x: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z.% \t])", "", x))
    return df

In [5]:
if parent_chain_use:
    # cleaning parent chain name as it has duplicated entries
    df = clean_text(data, parent_chain_column, '{}_{}'.format(parent_chain_column, 'norm'))
    # chain selection and columns to work on
    df_nlp = df[df['parent_chain_name_norm'] == parent_chain]
    df_nlp = df_nlp.loc[:, ['parent_chain_name_norm', item_column]].reset_index(drop=True)
else:
    df_nlp = data.loc[:, [item_column]].drop_duplicates().reset_index(drop=True)

In [6]:
# item name standardization
df_nlp.rename(columns={'sku_name': 'item_name'}, inplace=True)

In [7]:
print(f"Initial products: {len(list(set(df_nlp['item_name'].unique())))}")

Initial products: 4581


## 2. NLP Aplication

Natural Language Processing (NLP) is a procedure that let us synthesize natural language and speech. Steps:

###  2.1 Normalization

Remove noise so the computer can easier detect patterns.

In [8]:
df_nlp = clean_text(df_nlp, 'item_name', 'item_name_norm')

In [9]:
# comparing unique product names: pre-normalization vs post-normalization
len(df_nlp.item_name.unique()), len(df_nlp.item_name_norm.unique())

(4581, 4344)

### 2.2 Stop words

In [10]:
import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

if language_ == 'en':
    stop_words = stopwords.words('english')
elif language_ == 'es':
    stop_words = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
def replace_stop_words(df, col, stop_list):
    df['{}_stop'.format(col)] = df[col].apply(lambda x: ' '.join([word for word in x.split() if x not in stop_list]))
    return df

In [12]:
df_nlp = replace_stop_words(df_nlp, 'item_name_norm', stop_words)

### 2.3 Phrase tokenization

Tokenization refers to the process of splitting a phrase into a list, were each element of the list is a word of the phrase.

In [13]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
df_nlp['item_name_token'] = df_nlp['item_name_norm_stop'].apply(lambda x: word_tokenize(x))

### 2.4 Lemmatization

Here we aim to reduce each tokenized word to their root form. If we do this, we will reduce the noise in the dataset. Need to be careful because in the process we can also introduce duplicates.

Btw, the method used morphological analysis.

In [15]:
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/gonzalooportus/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [16]:
def word_lemmatizer(text):
    text_lemma = [WordNetLemmatizer().lemmatize(word) for word in text]
    return text_lemma

In [17]:
df_nlp['item_name_token_lemma'] = df_nlp['item_name_token'].apply(lambda x: word_lemmatizer(x))

### 2.5 Join Lemmas

We join each word of the sku_token_lemma list, by row, to get the root text.

In [18]:
df_nlp['product_name'] = df_nlp['item_name_token_lemma'].apply(lambda list_: ' '.join([word for word in list_]))

### 2.6 Clean product names with regex

In [19]:
regex_clean = r'(pm \d+\w+)|(pm \d+\.\d+)|(pm\d+\.\d+)|(\d+ pmp)|(pm\d+)|( \.+)|(pmp\d+.\d+)|(\d+pmp)|(pmp \d+)'

df_nlp['product_name'] = df_nlp['product_name'].apply(lambda x: re.sub(regex_clean, "", x))

NOTE: "df_nlp" will be the connection between the product name iterations and further columns with the raw data.

In [20]:
# unique items
len(df_nlp.item_name.unique()), len(df_nlp.product_name.unique())

(4581, 4291)

## 3. Fuzzywuzzy Ratios Calculation & Filtering

In this step, we apply the fuzzywuzzy method "token_sort_ratio" to identify product names similarities. What's Fuzzywuzzy? --> library that uses Levenshtein Distance to calculate the differences between sequences and patterns

In [21]:
# uselful columns selection (item selected just for 'product_name' not be treated as array)
df_fuzzy = df_nlp.loc[:, ['product_name']]
df_fuzzy = df_fuzzy.drop_duplicates('product_name').reset_index(drop=True)

QUIZAS PERMITE OPTIMIZAR SI HAGO UN MELT DEL CROSSTAB ANTES DE CALCULAR EL FUZZY RATIO

In [22]:
def fuzzy_ratios(df, item_name, method):
    """
    Calculates the fuzzy match between a set of products
    Inputs:
    - df: dataframe with all the data
    - item_name: product name column
    - method: fuzzywuzzy method to apply (ratio, partial_ratio, or sort_ratio)
    
    Returns a dataframe like: |product|match|fuzzy_ratio|
    """
    df_temp = df.copy()
    # fuzzy comparisson column
    df_temp['match'] = df_temp[item_name]
    # creating matrix
    cross_tab = pd.crosstab(df_temp[item_name], df_temp['match'])
    # calculating the fuzzy ratio between the product names
    cross_tab = cross_tab.apply(lambda col: [method(col.name, x) for x in col.index])
    # melting the matrix back to a pandas dataframe
    fuzzy_df = cross_tab.reset_index().melt(id_vars=[item_name], value_name='fuzz_ratio')
    # sort by: product name
    fuzzy_df = fuzzy_df.sort_values(by=item_name).reset_index(drop=True)
    return fuzzy_df

### 3.1 Calculation similarity ratios betwen products --> fuzz.token_sort_ratio() method

In [23]:
fuzzy = fuzzy_ratios(df=df_fuzzy, item_name='product_name', method=fuzz.token_sort_ratio)

In [24]:
# the result is a HUGE dataframe
fuzzy.shape

(18412681, 3)

In [25]:
fuzzy.head()

,product_name,match,fuzz_ratio
0,1.5 lt chardolini,1.5 lt chardolini,100
1,1.5 lt chardolini,milkybar 100g,27
2,1.5 lt chardolini,casillero del diablo cab sau 75cl,36
3,1.5 lt chardolini,skittle giant fruit sweet price marked bag 125g,34
4,1.5 lt chardolini,calypso strawberry lemonade light 16oz 473ml,36


### 3.2 Filtering matches by a similarity threshold

According to the literature, a threshold ratio of 80-85 for the fuzzy similarity ratio is a good filter. Below that means that the products don't share much information.

In [26]:
print(f'Threshold: {threshold_}')

Threshold: 82


In [27]:
# filtering the data by the default threshold
fuzzy_thl = fuzzy[fuzzy['fuzz_ratio'] > threshold_].reset_index(drop=True)

In [28]:
# filtering the dataframe above calms our mind down
fuzzy_thl.shape

(11057, 3)

In [29]:
# copy of the table to join
fuzzy_thl_copy = fuzzy_thl.copy()

In [30]:
# sorting just to keep my head "calm"
fuzzy_thl = fuzzy_thl.sort_values(by=['product_name', 'fuzz_ratio']).reset_index(drop=True)
fuzzy_thl_copy = fuzzy_thl_copy.sort_values(by=['product_name', 'fuzz_ratio']).reset_index(drop=True)

In [31]:
fuzzy_thl.head(10)

,product_name,match,fuzz_ratio
0,1.5 lt chardolini,1.5 lt chardolini,100
1,1.75l coca cola,coca cola 1.5l 1.,90
2,1.75l coca cola,coca cola 1.5ltr,90
3,1.75l coca cola,coca cola 1.75ltr,94
4,1.75l coca cola,coca cola 1.75,97
5,1.75l coca cola,1.75l coca cola,100
6,100 g buttery flapjack,100 g buttery flapjack,100
7,10pk carling can,10pk carling can,100
8,12 inch hot chicago town stuffed crust pizza p...,chicago town stuffed crust pepperoni pizza 490g,85
9,12 inch hot chicago town stuffed crust pizza p...,12 inch hot chicago town stuffed crust pizza p...,100


## 4. Extending product similarities

IDEA: Extend the products to which a product can be similar to. How? If product A is similar to B and C; and B is similar to F and G; and C is similar to A and D; then A must share relevant similarities with: B, C, D, F, and G.

### 4.1 Pre-work: join between the same table

In [32]:
# rename to create have clarity of columns when merging
fuzzy_thl_copy.rename(columns={'product_name': 'joined_by_match', 'match': 'extend_match', \
                               'fuzz_ratio': 'extend_fuzz_ratio'}, inplace=True)

In [33]:
merged_df = fuzzy_thl.merge(fuzzy_thl_copy, left_on='match', right_on='joined_by_match', how='inner')

In [34]:
merged_df.head()

,product_name,match,fuzz_ratio,joined_by_match,extend_match,extend_fuzz_ratio
0,1.5 lt chardolini,1.5 lt chardolini,100,1.5 lt chardolini,1.5 lt chardolini,100
1,1.75l coca cola,coca cola 1.5l 1.,90,coca cola 1.5l 1.,coca cola 1.75ltr,85
2,1.75l coca cola,coca cola 1.5l 1.,90,coca cola 1.5l 1.,coca cola 1.75,87
3,1.75l coca cola,coca cola 1.5l 1.,90,coca cola 1.5l 1.,coca cola 1.5ltr,88
4,1.75l coca cola,coca cola 1.5l 1.,90,coca cola 1.5l 1.,1.75l coca cola,90


### 4.2 Formatting the dataframe --> clarity

The fuzzy ratio of the candidate can be interpretated like this:

"The candidate A fuzzy similarity ratio with a product C, that is similar to B, and B is directly related to A, is the mean between the similarity of A-B and B-C".

In [35]:
# correcting the extend_fuzz_ratio measure
merged_df['extend_fuzz_ratio'] = merged_df[['fuzz_ratio', 'extend_fuzz_ratio']].mean(axis=1)

In [36]:
# removing duplicated column (equal to match)
merged_df.drop('joined_by_match', axis=1, inplace=True)

In [37]:
# all similars to same column (extending in number of rows): match & extend_match --> match
merged_df = merged_df.melt(id_vars=['product_name', 'fuzz_ratio', 'extend_fuzz_ratio'], value_name='candidate')
merged_df = merged_df.sort_values(by='product_name').reset_index(drop=True)

In [38]:
# keeping the correct ratio if variable is 'match' or 'extend_match' --> melt issue
merged_df['ratio'] = np.where(merged_df['variable'] == 'match', merged_df['fuzz_ratio'], \
                                                               merged_df['extend_fuzz_ratio'])

In [39]:
# droping extra columns
merged_df.drop(['fuzz_ratio', 'extend_fuzz_ratio'], axis=1, inplace=True)
merged_df.rename(columns={'ratio': 'fuzz_ratio'}, inplace=True)

In [40]:
# removing duplicated entries --> extend product is the same as the first match
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [41]:
merged_df.head()

,product_name,variable,candidate,fuzz_ratio
0,1.5 lt chardolini,match,1.5 lt chardolini,100.0
1,1.5 lt chardolini,extend_match,1.5 lt chardolini,100.0
2,1.75l coca cola,extend_match,1.75l coca cola,97.0
3,1.75l coca cola,extend_match,coca cola 1.75ltr,93.5
4,1.75l coca cola,extend_match,coca cola 1.5ltr,92.0


## 5. Identifying package similarity 

A useful method to explore a deeper level of relatedness.

### 5.1 Regex to identify the package

In [42]:
def package_extract(df, column, regex_):
    """
    Extracts the package from a product name. Uses a regular expression for these.
    
    Inputs:
    - df: dataframe
    - column: product name column where to look for packages
    - regex_: regular expression formula to match patterns
    
    Output: a column with the package of the specified product name column
    """
    packs = df[column].str.extract(regex_)
    packs['package'] = packs[packs.columns[0:]].apply(lambda x: ','.join(x.dropna()), axis=1)
    packs = packs.loc[:, ['package']]
    return packs.loc[:, ['package']]

NOTE: products like '7 up 500ml' have issues, or that contain 500ml or 500 ml (all products that end in ..0 something)

In [44]:
reg_package = r'(\d+x\d+\w+)|(\d+ x \d+\w+)|(\d+\.+\d+\w+)|(\d+\.+\d+ \w+)|(\d+ ml)|(\d+ g)|(\d+\w+)|(\d+ \w+)'

In [45]:
def clean_group(group_df, regex_, threshold_=85):
    """
    From a group of products which are similar, compares if they share the same/similar package. For this, uses 
    "package_extract" function to extract the package of the "leader" product, and its similars; and then uses
    fuzzywuzzy to compare the similarity of the packages. Finally, keeps the packages with similarity over a threshold
    of 75 or the one specified by the user.
    
    Inputs:
    - group_df: dataframe with a group of similar products
    - regex_: regex formula to extract the package
    - threshold_: threshold of similarity to compare package
    
    Output: a clean group of similar candidates
    """
    group_df['package'] = package_extract(group_df, 'product_name', regex_)
    group_df['package_candidate'] = package_extract(group_df, 'candidate', regex_)
    group_df['package_ratio'] = group_df.apply(lambda x: fuzz.token_sort_ratio(x['package'], x['package_candidate']), axis=1)
    group_df = group_df[group_df['package_ratio'] >= threshold_].copy()
    group_df = group_df.loc[:, ['product_name', 'candidate', 'fuzz_ratio']]
    group_df.reset_index(drop=True, inplace=True)
    return group_df


## 6. Identifying groups of products with high similarity

Idea: get groups of products that might be the same product (high similarity). Important to verify if the products are duplicated into more than a group.

Steps:
    
* Identify the product group
* Keep in the group products with similar package (fuzzy)
* Get every group candidate
* For each candidate:
    * Verify if is part of a group
    * Yes: concat all the products and re-create the group; remove the group from the catalog_df
    * No: keep the group as it is
    * Once finished for all, add the group with new ID

In [46]:
def group_selection(df, product):
    """
    Por a single product, identifies the products to which is related (all products in "candidate" column), and
    remove duplicated rows. The ratio is also included to keep the knowledge of the similarity ratio that let 
    them into the group.
    
    Inputs:
    - df: dataframe with threshold 80, with all matches and links (for all products)
    - product: the name of the product to work with
    
    Output: single product similarity group dataframe
    """
    # we select all the rows that match the product
    df_temp = df[df['product_name'] == product].copy()
    # remove duplicates: to see all the products to which a product is similar
    group = df_temp.drop_duplicates(subset=['product_name', 'candidate']).reset_index(drop=True)
    return group

In [47]:
def create_product_df(groups_df, product, product_group_list):
    
    if groups_df.shape[0] == 0:
        group_id = 0
    else:
        group_id = groups_df['group_id'].max() + 1
    df_temp = pd.DataFrame({
        'group_id': group_id,
        'leader': product,
        'member': product_group_list
        })
    return df_temp

In [48]:
def create_track_temp_df(track_df, applicants_list):
    if track_df.shape[0] == 0:
        group_id = 0
    else:
        group_id = track_df['group_id'].max() + 1
    df_temp = pd.DataFrame({
        'group_id': group_id,
        'member': applicants_list
        })
    return df_temp

In [49]:
def group_concat_procedure(groups_df, track_df, product_, applicants_list):
    # verify if any of the applicants is already assigned to a group, if not:
    if track_df[track_df['member'].isin(applicants_list)].shape[0] == 0:
        # create df for the group
        concat_df = create_product_df(groups_df, product_, applicants_list)
        # concat to the global groups df
        groups_df = pd.concat([groups_df, concat_df], axis=0).reset_index(drop=True)
        # create track applicants df
        df_track_temp = create_track_temp_df(track_df, applicants_list)
        # concat new group to track groups df
        track_df = pd.concat([track_df, df_track_temp], axis=0).reset_index(drop=True)
    else:
        # get the group ids where any of the candidates is assigned
        group_ids_list = list(track_df[track_df['member'].isin(applicants_list)]['group_id'].unique())
        # locate where the group is
        select_df = groups_df[groups_df['group_id'].isin(group_ids_list)]
        # list of actual members of the group
        already_members = list(pd.unique(select_df[['leader', 'member']].values.ravel('K')))
        # union of already members + apliccants list --> idea: get a unique selection of a wider spectrum
        concatenated_list = list(set(already_members + applicants_list))
        # remove group from global groups dataframe
        groups_df = groups_df[~groups_df['group_id'].isin(group_ids_list)].copy()
        # remove group from track groups dataframe
        track_df = track_df[~track_df['group_id'].isin(group_ids_list)]
        # re-create and add the modified group to the global groups df
        concat_df = create_product_df(groups_df, product_, concatenated_list)
        groups_df = pd.concat([groups_df, concat_df], axis=0).reset_index(drop=True)
        # re-create and add the modified group to the track groups df
        df_track_temp = create_track_temp_df(track_df, applicants_list)
        track_df = pd.concat([track_df, df_track_temp], axis=0).reset_index(drop=True)
    return groups_df, track_df

In [50]:
merged_df.drop('variable', axis=1, inplace=True)

### 6.1 Selecting the similarity group of a product

In [51]:
# dataframe definition
groups_df = pd.DataFrame(columns=['group_id', 'leader', 'member'])
track_df = pd.DataFrame(columns=['group_id', 'member'])

In [52]:
# all products 
products_list = merged_df['product_name'].unique()

In [53]:
for product_ in products_list:
    # select the group of similars to product_
    candidate_group = group_selection(merged_df, product_)
    # remove products without package similarity (fuzzy: threshold specified)
    clean_candidate_group = clean_group(candidate_group, reg_package, threshold_=85)
    # get all the products in the applicant group
    applicants_list = list(pd.unique(clean_candidate_group[['product_name', 'candidate']].values.ravel('K')))
    # integration to global groups df
    groups_df, track_df = group_concat_procedure(groups_df, track_df, product_, applicants_list)

In [54]:
groups_df

,group_id,leader,member
0,0,1.5 lt chardolini,1.5 lt chardolini
1,2,100 g buttery flapjack,100 g buttery flapjack
2,3,10pk carling can,10pk carling can
3,4,12 inch hot chicago town stuffed crust pizza p...,12 inch hot chicago town stuffed crust pizza p...
4,5,12 pizza meal deal,12 pizza meal deal
...,...,...,...
4306,3818,young admiral pie 300g,young admiral pie 300g
4307,3819,yp pea spinach soup 600g,yp pea spinach soup 600g
4308,3820,zoflora 3in1 disinfectant 120ml,zoflora 3in1 disinfectant 120ml
4309,3821,zyweic 4 pack,zyweic 4 pack


In [55]:
track_df

,group_id,member
0,0,1.5 lt chardolini
1,2,100 g buttery flapjack
2,3,10pk carling can
3,4,12 inch hot chicago town stuffed crust pizza p...
4,5,12 pizza meal deal
...,...,...
4142,3818,young admiral pie 300g
4143,3819,yp pea spinach soup 600g
4144,3820,zoflora 3in1 disinfectant 120ml
4145,3821,zyweic 4 pack


## 9. Outputs

In [71]:
groups_df.to_csv('product_groups_{}.csv'.format(threshold_), index=False)